In [1]:
import geopandas as gpd 
from bs4 import BeautifulSoup
import folium 
from geopy.geocoders import Nominatim

file = open('/Users/beilinteo/Desktop/Tableau Fun Times /lta-bicycle-rack/lta-bicycle-rack-geojson.geojson')
df = gpd.read_file(file)

In [44]:
#Preparing the Data 

des=df['Description']
geo=df['geometry']

shelter = []
type = []
rack_count = []
add = []

for s in des:
    soup = BeautifulSoup(s)
    table = soup.find('table')
    dataset = [td.get_text() for td in table.find_all('td')[:3]]
    shelter.append(dataset[0])
    type.append(dataset[1])
    rack_count.append(dataset[2])

geolocator = Nominatim(user_agent="openstreetmaps")

for g in geo: 
    coord = g.y, g.x
    location = geolocator.reverse(coord)
    add.append(location.address)

    
df['Shelter']= shelter
df['Type']= type
df['Rack_Count']= rack_count
df['Address']=add 


In [43]:
#Creating the map 

m = folium.Map(location=[1.3521, 103.8198],zoom_start=12, tiles='CartoDB positron')
   
 
for grp_name, df_grp in df.groupby('Type'):
    feature_group=folium.FeatureGroup(grp_name)

    for row in df_grp.itertuples():
        popup = folium.Popup("<b>Address: </b>"+row.Address+"<br><b>Shelter: </b>"+row.Shelter+"<br><b>Rack Count: </b>"+row.Rack_Count, max_width=300, min_width=300)
        
        if row.Type=='Single':
            folium.CircleMarker(
                location=[row.geometry.y, row.geometry.x],
                popup=popup,
                radius=int(row.Rack_Count)/6,
                stroke=False,
                color='pink',
                fill_color='pink',
                fill_opacity=.7,
                fill=True
            ).add_to(feature_group)
       
        elif row.Type=='Double':
            folium.CircleMarker(
                location=[row.geometry.y, row.geometry.x],
                popup=popup,  
                radius=int(row.Rack_Count)/6,
                stroke=False,
                color='cadetblue',
                fill_color='cadetblue',
                fill_opacity=.7,
                fill=True
            ).add_to(feature_group)    
    
        else:
            folium.CircleMarker(
                location=[row.geometry.y, row.geometry.x],
                popup=popup, 
                radius=10,
                stroke=False,
                color='orange',
                fill_color='orange',
                fill_opacity=.7,
                fill=True
            ).add_to(feature_group)    
            
    feature_group.add_to(m)

    
folium.LayerControl(position='bottomleft',collapsed=False).add_to(m)


m.save('bikerack_map.html')